# Selective Acquisition using Leica CAM Server

### About 

This is a basic example for selective acquisition using the Leica CAM Server and Python.


### Requirements

* **Python 2.7**: the cam_communicator_class used is still based on the soon-to-be-outdated python 2.7
* skimage
* toolz

### Target audience

You will require some basic python skills to adapt this to your needs.

### Author
Volker Hilsenstein, October 2019


In [50]:
import pathlib
import tifffile
import skimage
import re
import numpy as np
from functools import partial
from toolz.dicttoolz import merge
from toolz.itertoolz import take
from toolz.itertoolz import groupby
import cam_communicator_class as CAMC
import os.path

## Establish connection to Leica CAM Server

On the microscope, you need to have Matrix Screener running, the CAM Server checkbox ticked and at least one scan field assigned:

In [31]:
sp5=CAMC.CAMcommunicator()

In [32]:
sp5.setIP('127.0.0.1')

In [33]:
sp5.open()

Trying to open connection to Leica at  127.0.0.1 : 8895
Connected.


True

Flush any old and stale messages

In [34]:
sp5.flushCAMreceivebuffer()

define a function that finds the last experiment folder that matrix screener created:

In [55]:
def get_last_experimentfolder(basepath):
    files = basepath.glob("experiment*")
    files = map(str, files)
    files.sort()
    return(files[-1])

In [56]:
basepath = pathlib.Path("E:/22-07-2019/4927/")
#rootfolder =  pathlib.Path("E:/22-07-2019/4927/experiment--2019_07_22_05_07_51")

define a regular expression that lets us extract metadata matrix screener filenames

In [20]:
regex = r"^image--L\d+--S\d+--U(?P<u>\d+)--V(?P<v>\d+)"\
    "--J(?P<jobnr>\d+).*--X(?P<x>\d+)--Y(?P<y>\d+)--T(?P<t>\d+)" \
    "--Z(?P<z>\d+)--C(?P<ch>\d+).ome.tif"

In [21]:
regex

'^image--L\\d+--S\\d+--U(?P<u>\\d+)--V(?P<v>\\d+)--J(?P<jobnr>\\d+).*--X(?P<x>\\d+)--Y(?P<y>\\d+)--T(?P<t>\\d+)--Z(?P<z>\\d+)--C(?P<ch>\\d+).ome.tif'

Define some helper functions for:
* measuring image stats
* extracting metadata 

In [58]:
def measure_image_stats(impath):
    filename = str(impath)
    print(filename)
    im = tifffile.imread(filename)
    
    stats = {}
    # there really should be a numpy 
    # function that finds both min and max
    # with a single traversal
    # this would maybe be something for dask-image as
    # well
    stats["max"] = np.max(im)
    stats["min"] = np.min(im)
    stats["median"] = np.median(im)
    stats["sum"] = np.sum(im)
    return(stats)

def extraxt_meta(impath, regex):
    mobj = re.match(regex, str(impath.name))
    return mobj.groupdict()

def call_and_merge(argument, funcs):
    return (merge(map(lambda f: f(argument), funcs)))

def coords_from_dict(d):
    return d["u"],d["v"],d["x"],d["y"]

In [59]:
meta = partial(extraxt_meta, regex=regex)
cm = partial(call_and_merge, funcs=[meta, measure_image_stats])

After setting up all the helper functions and establishing a connection, the following cell starts selective acquisition. It is assumed that the prescan job is already assigned to the scan fields.

* starts the initial scan with the already assigned prescan job
* waits for the scan to finish
* figures out which folder matrix screener wrote the files to (finding the most recent folder)
* Assigns the high resolution scanjob (harcoded as 'acquisition') to all fields.
* Disables all scanfields.
* finds all the files that were created and computes image statistics 
* goes through all fields and looks whether any of the zslices exceeds a certain minimum brightness (hardcoded as 20 here). If so, the respective scanfield is enabled
* starts the high-resolution scan

In [ ]:
print("Starting the prescan")
sp5.startScan()
print("Waiting for prescan to finish")
sp5.waitForScanToFinish()
rootfolder = pathlib.Path(get_last_experimentfolder(basepath))

# Assign acquisition job to all fields and then disable all fields
sp5.selectScanField(allfields=True)
sp5.assignJob("acquisition")
sp5.disableScanField(allfields=True)

# Find and process all files from prescan 
tifffiles = rootfolder.rglob("*.tif")
results = map(cm, tifffiles)
zstacks = groupby(coords_from_dict, results)
# this could be done in a functional way, too ... 
# using some sort of reduce to find the maximum in each stack
# and filter to filter the stacks
# I just happened to be in a rush ... so I did it the old-fashioned way
for key in zstacks.keys():
    maxima = list(map(lambda x: x["max"], zstacks[key]))
    print(maxima)
    if np.max(maxima) > 20:
        print(str(key) + " is bright enough ++++++++")
        u,v, x,y = [int(elem)+1 for elem in key]
        print("enabling field ",u,v,x,y)
        sp5.enableScanField(wellx=u, welly=v, fieldx=x, fieldy=y)
    else:
        print(str(key) + " too dim --")
        
print("starting the acquisition")
sp5.startScan()

Starting the prescan
Waiting for prescan to finish
